In [5]:
!pip install feature_engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 5.5 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from feature_engine.outliers import Winsorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

import sklearn.metrics as skmet
import pickle

In [4]:
data = pd.read_csv("/content/cancerdata.csv")

In [6]:
data.head(3)

,id,diagnosis,Sex,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,87139402,B,F,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.1242,0.09391,0.2827,0.06771
1,8910251,B,M,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.1916,0.07926,0.2940,0.07587
2,905520,B,M,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.1067,0.07431,0.2998,0.07881


In [7]:
data['diagnosis'] = np.where(data['diagnosis'] == 'B', 'Benign', data['diagnosis'])
data['diagnosis'] = np.where(data['diagnosis'] == 'M', 'Malignant', data['diagnosis'])

In [8]:
data.drop(['id'], axis = 1, inplace = True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   diagnosis          569 non-null    object 
 1   Sex                569 non-null    object 
 2   radius_mean        569 non-null    float64
 3   texture_mean       569 non-null    float64
 4   perimeter_mean     569 non-null    float64
 5   area_mean          569 non-null    float64
 6   smoothness_mean    569 non-null    float64
 7   compactness_mean   569 non-null    float64
 8   concavity_mean     567 non-null    float64
 9   points_mean        568 non-null    float64
 10  symmetry_mean      568 non-null    float64
 11  dimension_mean     569 non-null    float64
 12  radius_se          569 non-null    float64
 13  texture_se         569 non-null    float64
 14  perimeter_se       569 non-null    float64
 15  area_se            569 non-null    float64
 16  smoothness_se      569 non

In [11]:
num_feat = data.select_dtypes(exclude = ['object']).columns
num_feat

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean',
       'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'points_se', 'symmetry_se', 'dimension_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'points_worst', 'symmetry_worst', 'dimension_worst'],
      dtype='object')

In [14]:
num_pipeline = Pipeline([('impute', SimpleImputer(strategy = 'mean'))])

In [15]:
cat_feat = ['Sex']
cat_pipeline=Pipeline([('label', DataFrameMapper([(cat_feat, OneHotEncoder(drop = 'if_binary'))]))])

In [16]:
preprocess_pipeline = ColumnTransformer([('categorical', cat_pipeline, cat_feat),('numerical',num_pipeline,num_feat)])

In [18]:
processed = preprocess_pipeline.fit(data)

ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('label',
                                                  DataFrameMapper(drop_cols=[],
                                                                  features=[(['Sex'],
                                                                             OneHotEncoder(drop='if_binary'))]))]),
                                 ['Sex']),
                                ('numerical',
                                 Pipeline(steps=[('impute', SimpleImputer())]),
                                 Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean', 'point...
       'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'points_se', 'symmetry_se', 'dimension_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'points_worst', 'symmetry_worst', 'dimension_worst'],
      dtype='object'))])

In [19]:
import joblib
joblib.dump(processed, 'processed_cancer')

['processed_cancer']

In [20]:
wbcd = pd.DataFrame(processed.transform(data))

In [21]:
wbcd

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,1.0,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,1.0,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,0.0,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,0.0,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.0,13.17,18.22,84.28,537.3,0.07466,0.05994,0.04859,0.02870,0.1454,...,14.90,23.89,95.10,687.6,0.1282,0.1965,0.18760,0.10450,0.2235,0.06925
565,1.0,10.26,14.71,66.20,321.6,0.09882,0.09159,0.03581,0.02037,0.1633,...,10.88,19.48,70.89,357.1,0.1360,0.1636,0.07162,0.04074,0.2434,0.08488
566,0.0,15.28,22.41,98.92,710.6,0.09057,0.10520,0.05375,0.03263,0.1727,...,17.80,28.03,113.80,973.1,0.1301,0.3299,0.36300,0.12260,0.3175,0.09772
567,1.0,14.53,13.98,93.86,644.2,0.10990,0.09242,0.06895,0.06495,0.1650,...,15.80,16.93,103.10,749.9,0.1347,0.1478,0.13730,0.10690,0.2606,0.07810


In [22]:
wbcd.columns

RangeIndex(start=0, stop=31, step=1)

In [24]:
new_feat = wbcd.select_dtypes(exclude = ['object']).columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64')

In [25]:
scale_pipeline = Pipeline([('scale', MinMaxScaler())])

In [27]:
prep_pipeline2 = ColumnTransformer([('scale',scale_pipeline, new_feat)], remainder = 'passthrough')
processed2 = prep_pipeline2.fit(wbcd)

ColumnTransformer(remainder='passthrough',
                  transformers=[('scale',
                                 Pipeline(steps=[('scale', MinMaxScaler())]),
                                 Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64'))])

In [28]:
joblib.dump(processed2, 'processed2_cancer')

['processed2_cancer']

In [30]:
wbcd_scaled = pd.DataFrame(processed2.transform(wbcd))

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,0.252686,0.090632,0.242278,0.135992,0.452920,0.154684,0.093416,0.183897,0.454040,...,0.198150,0.096482,0.182081,0.089437,0.444628,0.096351,0.099201,0.322715,0.248768,0.083104
1,1.0,0.171281,0.312479,0.176145,0.086066,0.399476,0.292375,0.149649,0.131312,0.435354,...,0.140519,0.291045,0.138802,0.058887,0.331044,0.217530,0.153035,0.272371,0.271043,0.136626
2,1.0,0.192106,0.240785,0.187478,0.097434,0.497156,0.179928,0.071368,0.123260,0.330303,...,0.159374,0.384328,0.147019,0.070340,0.434062,0.117307,0.085224,0.255361,0.282476,0.155910
3,0.0,0.203464,0.124450,0.201852,0.102354,0.575697,0.289001,0.108599,0.238370,0.359091,...,0.141942,0.099947,0.130086,0.061148,0.432741,0.150294,0.069241,0.295911,0.105855,0.083956
4,0.0,0.388518,0.118363,0.372193,0.241060,0.243748,0.153242,0.079499,0.132058,0.333838,...,0.294201,0.098881,0.269386,0.155795,0.273592,0.142048,0.108786,0.281031,0.181747,0.082776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.0,0.292915,0.287792,0.279801,0.167041,0.198881,0.124410,0.113847,0.142644,0.198990,...,0.247954,0.316365,0.222571,0.123476,0.376610,0.164168,0.149840,0.359107,0.132072,0.093205
565,1.0,0.155190,0.169090,0.154861,0.075546,0.416990,0.221489,0.083903,0.101243,0.289394,...,0.104945,0.198827,0.101997,0.042248,0.428119,0.132249,0.057204,0.140000,0.171299,0.195723
566,0.0,0.392778,0.429489,0.380969,0.240551,0.342512,0.263235,0.125937,0.162177,0.336869,...,0.351121,0.426706,0.315703,0.193644,0.389157,0.293594,0.289936,0.421306,0.317366,0.279942
567,1.0,0.357281,0.144403,0.346002,0.212386,0.517017,0.224035,0.161551,0.322813,0.297980,...,0.279972,0.130864,0.262413,0.138788,0.419534,0.116919,0.109665,0.367354,0.205204,0.151253


In [31]:
#Seperating ip and op
X = np.array(wbcd_scaled.iloc[:,:])#Predictors
Y = np.array(data['diagnosis'])#Target

In [32]:
X_train, X_test,Y_train, Y_test = train_test_split(X,Y , test_size = 0.2, random_state = 0)

In [33]:
knn = KNeighborsClassifier(n_neighbors = 20)
KNN = knn.fit(X_train, Y_train)

In [34]:
pred_train = knn.predict(X_train)

In [35]:
pd.crosstab(Y_train, pred_train, rownames = ['Actual'], colnames = ['Predictions'])

Predictions,Benign,Malignant
Actual,,
Benign,280,0
Malignant,18,157


In [36]:
print(skmet.accuracy_score(Y_train, pred_train))

0.9604395604395605


In [37]:
pred_test = knn.predict(X_test)

In [39]:
print(skmet.accuracy_score(Y_test,pred_test))

0.9122807017543859


In [40]:
acc=[]
for i in range(3,50,2):
  neighbor = KNeighborsClassifier(n_neighbors =i)
  neighbor.fit(X_train, Y_train)
  train_acc= np.mean(neighbor.predict(X_train) == Y_train)
  test_acc = np.mean(neighbor.predict(X_test) == Y_test)
  diff = train_acc - test_acc
  acc.append([diff,train_acc, test_acc])



In [41]:
acc

[[0.021881627144785032, 0.978021978021978, 0.956140350877193],
 [0.026238673607094642, 0.9648351648351648, 0.9385964912280702],
 [0.017447464815885882, 0.9560439560439561, 0.9385964912280702],
 [0.010854058222479201, 0.9494505494505494, 0.9385964912280702],
 [0.037189126662810934, 0.9582417582417583, 0.9210526315789473],
 [0.041584731058415314, 0.9626373626373627, 0.9210526315789473],
 [0.041584731058415314, 0.9626373626373627, 0.9210526315789473],
 [0.045961056487372365, 0.9582417582417583, 0.9122807017543859],
 [0.034991324465008744, 0.9560439560439561, 0.9210526315789473],
 [0.039386928860613124, 0.9604395604395605, 0.9210526315789473],
 [0.039386928860613124, 0.9604395604395605, 0.9210526315789473],
 [0.039386928860613124, 0.9604395604395605, 0.9210526315789473],
 [0.043763254289570175, 0.9560439560439561, 0.9122807017543859],
 [0.061307113938692925, 0.9560439560439561, 0.8947368421052632],
 [0.052535184114131495, 0.9560439560439561, 0.9035087719298246],
 [0.050337381916329305, 0.9